In [1]:
pip install numpy pandas pillow torch torchvision scikit-learn matplotlib --break-system-packages



   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.4/241.3 MB 7.6 MB/s eta 0:00:32
   ---------------------------------------- 0.9/241.3 MB 9.7 MB/s eta 0:00:25
   ---------------------------------------- 1.5/241.3 MB 10.4 MB/s eta 0:00:24
   ---------------------------------------- 1.9/241.3 MB 11.3 MB/s eta 0:00:22
   ---------------------------------------- 2.3/241.3 MB 10.7 MB/s eta 0:00:23
   ---------------------------------------- 2.7/241.3 MB 10.0 MB/s eta 0:00:24
   ---------------------------------------- 2.9/241.3 MB 8.9 MB/s eta 0:00:27
    --------------------------------------- 3.2/241.3 MB 8.9 MB/s eta 0:00:27
    --------------------------------------- 3.5/241.3 MB 8.3 MB/s eta 0:00:29
    --------------------------------------- 3.9/241.3 MB 8.5 MB/s eta 0:00:28
    --------------------------------------- 4.2/241.3 MB 8.5 MB/s eta 0:00:28
    --------------------------------------- 4.7/241.3 MB 8.5 MB/s 

In [2]:
# 3D-Print Defect Detection from CSV — ResNet-9 (scratch)
# Train on images/all_images256, Test on images/test_images_oblique256 + test_images_silver265 (oblique=กล้องรอง,silver=กล้องรอง )
# CSVs: general_data/all_images_no_filter.csv (train), general_data/all_images_no_filter.csv or specific test CSVs if needed
# Works on ROCm/NVIDIA automatically. (AKE=7800xt,i5 13500H)

import os, random, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
from torchvision.ops.misc import Conv2dNormActivation

# ========= Repro & Device =========
SEED = 1337
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.set_per_process_memory_fraction(0.95, device=0)



Device: cpu


In [3]:
import resource

# คำนวณ 95% ของ RAM ทั้งหมด
total_ram = resource.getrlimit(resource.RLIMIT_AS)[1]
if total_ram == resource.RLIM_INFINITY:
    import psutil
    total_ram = psutil.virtual_memory().total

limit = int(total_ram * 0.95)  # 95% ของ RAM

# ตั้งค่าขีดจำกัดการใช้ RAM ของโปรแกรม
resource.setrlimit(resource.RLIMIT_AS, (limit, limit))


ModuleNotFoundError: No module named 'resource'

In [4]:

# ========= Paths (ปรับตามโครงสร้างในภาพของคุณ) =========
ROOT = Path("/home/ake/Desktop/CNN/Printing_Errors")
CSV_DIR = ROOT / "general_data"
IMG_ROOT = ROOT / "images"

TRAIN_IMG_DIR = IMG_ROOT / "all_images256"
TEST_IMG_DIRS = [IMG_ROOT / "test_images_oblique256", IMG_ROOT / "test_images_silver265"]

TRAIN_CSV = CSV_DIR / "all_images_no_filter.csv"     # รวมรูป train ทั้งหมด
# ถ้าคุณมี CSV test แยก ก็ใส่ได้ เช่น CSV_DIR/"black_bed_all.csv"; แต่ในตัวอย่างนี้จะสร้าง test จากโฟลเดอร์โดยฟิลเตอร์จาก TRAIN_CSV

# ========= Helper: โหลด CSV ให้ทนทานต่อ ',' หรือ ';' =========
def read_any_csv(path: Path):
    # engine='python' + sep=None => ให้ pandas เดา delimiter อัตโนมัติ
    return pd.read_csv(path, sep=None, engine="python")
# ========= NEW: ตรวจสอบว่า TRAIN_CSV มีอยู่จริงไหม =========
if not TRAIN_CSV.exists():
    print(f"\n❌ ไม่พบไฟล์: {TRAIN_CSV.resolve()}")
    print("📂 ลองดูไฟล์ .csv ที่มีอยู่ในโฟลเดอร์นี้แทน:\n")

    # แสดงไฟล์ .csv ทั้งหมดในโฟลเดอร์
    csv_files = list(CSV_DIR.glob("*.csv"))
    if csv_files:
        for f in csv_files:
            print(" -", f.name)
    else:
        print("⚠️ ไม่มีไฟล์ .csv อยู่ในโฟลเดอร์นี้เลย")
    
    # หยุดการทำงานเพื่อป้องกัน error ต่อ
    raise FileNotFoundError("⛔ หยุด: ไม่พบไฟล์ TRAIN_CSV")
else:
    print(f"✅ พบไฟล์ TRAIN_CSV: {TRAIN_CSV.name}")
# ========= Dataset จาก CSV =========
class CSVDataset(Dataset):
    def __init__(self, df: pd.DataFrame, img_dir: Path, transform=None):
        """
        df ต้องมีคอลัมน์: 'image' (ชื่อไฟล์), 'class' (0,1,2,4)
        img_dir คือโฟลเดอร์ที่ไฟล์ภาพของ df นี้อยู่
        """
        self.df = df.reset_index(drop=True).copy()
        self.img_dir = Path(img_dir)
        self.transform = transform
        # sanitize class to int
        self.df["class"] = self.df["class"].astype(int)

        # เก็บเฉพาะรูปที่มีอยู่จริงในโฟลเดอร์เป้าหมาย
        paths, labels = [], []
        for _, r in self.df.iterrows():
            p = self.img_dir / str(r["image"])
            if p.is_file():  # ถ้ามีไฟล์จริงค่อยใช้
                paths.append(p)
                labels.append(int(r["class"]))
        self.paths = paths
        self.labels = labels

        # map index->raw label (0,1,2,4)
        self.classes_raw = sorted(list(set(self.labels)))
        self.raw2idx = {raw:i for i, raw in enumerate(self.classes_raw)}
        self.idx2raw = {i:raw for raw, i in self.raw2idx.items()}

        # แปลงเป็น index ที่ต่อเนื่องสำหรับ CrossEntropyLoss
        self.labels_idx = [self.raw2idx[c] for c in self.labels]

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        y_idx = self.labels_idx[idx]
        if self.transform: img = self.transform(img)
        return img, y_idx


✅ พบไฟล์ TRAIN_CSV: all_images_no_filter.csv


In [5]:

# ========= Load CSV =========
df_all = read_any_csv(TRAIN_CSV)

# -------- Train set: all_images256 --------
df_train = df_all.copy()
ds_train = CSVDataset(df_train, TRAIN_IMG_DIR,
                      transform=transforms.Compose([
                          transforms.Resize((256,256)),
                          transforms.RandomHorizontalFlip(),
                          transforms.RandomVerticalFlip(p=0.2),
                          transforms.RandomRotation(15),
                          transforms.ColorJitter(brightness=0.1, contrast=0.1),
                          transforms.ToTensor(),
                      ]))

# ใช้ mapping class จาก train เป็นมาตรฐาน
idx2raw = [ds_train.idx2raw[i] for i in range(len(ds_train.idx2raw))]
raw2idx = {raw:i for i, raw in enumerate(idx2raw)}
num_classes = len(idx2raw)
print("Train classes (index->raw):", {i:r for i,r in enumerate(idx2raw)})

# -------- Test set: รวมโฟลเดอร์ oblique256 + silver265 โดยใช้บรรทัดจาก CSV เดิมเพื่อดึง label --------
def build_test_dataset_from_dirs(df_all: pd.DataFrame, test_dirs, transform):
    # รวมชื่อไฟล์ที่มีอยู่จริงใน test_dirs ทั้งหมด
    test_files = set()
    for d in test_dirs:
        d = Path(d)
        for p in d.glob("*"):
            if p.is_file(): test_files.add(p.name)

    # จับคู่ชื่อไฟล์กับข้อมูลใน CSV (หา label จาก df_all)
    df_test = df_all[df_all["image"].isin(test_files)].copy()
    return CSVDataset(df_test, test_dirs[0].parent, transform)  # ใช้ parent (images/) แล้ว CSVDataset จะเช็ค path ภายในโฟลเดอร์ที่ส่งมา
    # หมายเหตุ: เราสร้าง CSVDataset ทีละโฟลเดอร์ก็ได้ แต่เพื่อความง่าย ใช้ภาพที่อยู่จริงในแต่ละโฟลเดอร์ผ่าน __init__ แล้วกรองเอง

ds_test = build_test_dataset_from_dirs(
    df_all,
    TEST_IMG_DIRS,
    transform=transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()])
)

# ทำให้ label ของ test ตรงกับ index ของ train
def remap_labels_to_train(ds, raw2idx):
    # remap labels_idx ตาม raw2idx ของ train
    new_labels_idx = []
    kept_paths = []
    for p, raw_label in zip(ds.paths, ds.labels):
        if raw_label in raw2idx:
            new_labels_idx.append(raw2idx[raw_label]); kept_paths.append(p)
    ds.paths = kept_paths
    ds.labels_idx = new_labels_idx
    ds.classes_raw = [idx2raw[i] for i in range(len(idx2raw))]
    ds.raw2idx = raw2idx
    ds.idx2raw = {i:r for i,r in enumerate(idx2raw)}
    return ds

ds_test = remap_labels_to_train(ds_test, raw2idx)
print("Test samples:", len(ds_test))


Train classes (index->raw): {0: 0, 1: 1, 2: 2, 3: 4}
Test samples: 0


In [6]:

# ========= DataLoaders =========
BATCH = 32
kwargs = dict(batch_size=BATCH, num_workers=2, pin_memory=torch.cuda.is_available())
train_ld = DataLoader(ds_train, shuffle=True,  **kwargs)
test_ld  = DataLoader(ds_test,  shuffle=False, **kwargs)


In [7]:

# ========= Model: ResNet-9 =========
def conv_bn_relu(in_c, out_c, k=3, s=1, p=1):
    return Conv2dNormActivation(in_c, out_c, kernel_size=k, stride=s, padding=p,
                                norm_layer=nn.BatchNorm2d, activation_layer=nn.ReLU)

class BasicBlock(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.conv1 = conv_bn_relu(c, c)
        self.conv2 = Conv2dNormActivation(c, c, kernel_size=3, padding=1,
                                          norm_layer=nn.BatchNorm2d, activation_layer=None)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        id = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = x + id
        return self.relu(x)

class ResNet9(nn.Module):
    def __init__(self, in_ch=3, num_classes=4):
        super().__init__()
        self.layer1 = conv_bn_relu(in_ch, 64, 3, 1, 1)
        self.layer2 = conv_bn_relu(64, 128, 3, 2, 1)  # /2
        self.res1   = BasicBlock(128)
        self.layer3 = conv_bn_relu(128, 256, 3, 2, 1) # /4
        self.layer4 = conv_bn_relu(256, 512, 3, 2, 1) # /8
        self.res2   = BasicBlock(512)
        self.pool   = nn.AdaptiveAvgPool2d(1)
        self.fc     = nn.Linear(512, num_classes)
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x); x = self.res1(x)
        x = self.layer3(x)
        x = self.layer4(x); x = self.res2(x)
        x = self.pool(x).flatten(1)
        return self.fc(x)

model = ResNet9(num_classes=num_classes).to(device)


In [ ]:

# ========= Train (no valid set) =========
EPOCHS = 30
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
use_amp = torch.cuda.is_available()
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

for ep in range(1, EPOCHS+1):
    model.train()
    total, correct, run_loss = 0, 0, 0.0
    for x, y in train_ld:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=use_amp):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        run_loss += loss.item()*x.size(0)
        correct += (logits.argmax(1)==y).sum().item()
        total   += x.size(0)
    print(f"Epoch {ep:02d}/{EPOCHS}  loss {run_loss/total:.4f}  acc {correct/total:.3f}")

In [ ]:

# ========= Test score + Confusion Matrix =========
model.eval()
y_true, y_pred = [], []
with torch.inference_mode(), torch.cuda.amp.autocast(enabled=use_amp):
    for x, y in test_ld:
        x = x.to(device, non_blocking=True)
        logits = model(x)
        y_true.extend(y.numpy().tolist())
        y_pred.extend(logits.argmax(1).cpu().numpy().tolist())

acc = accuracy_score(y_true, y_pred)
print(f"\n=== TEST SCORE ===\nAccuracy: {acc:.4f}")
cm = confusion_matrix(y_true, y_pred, labels=list(range(num_classes)))

fig, ax = plt.subplots(figsize=(6,5), dpi=150)
ConfusionMatrixDisplay(cm, display_labels=[str(r) for r in idx2raw]).plot(ax=ax, cmap="Blues", values_format="d", colorbar=False)
plt.title("Confusion Matrix (counts)"); plt.tight_layout()
plt.setp(ax.get_xticklabels(), rotation=90); plt.show()

cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)
fig, ax = plt.subplots(figsize=(6,5), dpi=150)
ConfusionMatrixDisplay(cm_norm, display_labels=[str(r) for r in idx2raw]).plot(ax=ax, cmap="Blues", values_format=".2f", colorbar=True)
plt.title("Confusion Matrix (row-normalized)"); plt.tight_layout()
plt.setp(ax.get_xticklabels(), rotation=90); plt.show()
